In [81]:
import json
import math
import argparse
import logging
from typing import Literal, NoReturn

import pandas as pd # type: ignore
import numpy as np # type: ignore
from scipy.interpolate import interp1d # type: ignore

## Load Data

In [2]:
additionality = pd.read_csv("/maps/jh2589/tmf_pipe_out/redd/additionality/1201-additionality.csv", index_col="year")
leakage = pd.read_csv("/maps/jh2589/tmf_pipe_out/redd/leakage/1201-leakage.csv", index_col="year")
scc = pd.read_csv("/home/jh2589/tmf-implementation/scc_extended.csv")
scc = scc[['year', 'central']]

### Define Net DF Function

The `net_additionality` function calculates the net additionality by merging the `additionality` and `leakage` dataframes on the 'year' column. It then computes the net value for each year by summing the additionality and leakage values. The function also calculates the yearly change in net additionality and a rolling average of these changes over a specified period. The first eleven rows are dropped to remove data from before the project started. The resulting dataframe includes columns for the year (`year`), additionality (`additionality`), leakage (`leakage`), net additionality (`net`), year on year change in net additionality (`year_change`), and the subsequent rolling average of year changes (`year_avg`). For this latter metric, the `period` for the rolling averages is set any time frame deemed suitable. In the PACT methodology this is 5 years. The last row value for `year_avg` is equivalent to the *R* rate in PACT.

In [3]:
def net_additionality(additionality, leakage, period):
    # Merge on the 'year' column
    net_df = pd.merge(additionality, leakage, on='year')

    # Calculate net value (additionality + leakage) for each year
    net_df['net'] = net_df['additionality'] + net_df['leakage']

    # Add yearly change column
    net_df['yearly_change'] = net_df['net'].diff()

    # Drop the first eleven rows (these have data from prior to the project)
    net_df = net_df.iloc[11:].reset_index(drop=False)

    # Calculate rolling average yearly change in net additionaltiy
    net_df['year_avg'] = net_df['yearly_change'].rolling(window=period).mean()

    return net_df

### Define Forecaster Function

The `forecaster` function projects the net additionality values into the future until a specified project end year. It starts by extracting the last row's values from the `net_df` dataframe, including the year, net additionality, and the rolling average of yearly changes. The function then grows the net value by the yearly average change until the project end year. After the project end year, it shrinks the net value by the yearly change until it hits zero. The function ensures the last value is zero and adjusts the last yearly change accordingly. The resulting dataframe includes columns for the year (`year`), net additionality (`net`), yearly change in net additionality (`year_change`), and the social cost of carbon discount (`scc_discount`). The `scc_discount` is calculated for each year using the `scc` dataframe and the specified discount rate (`delta`). In the PACT Metholodogy, this is set at 3%

In [99]:
def forecaster(net_df, project_end, scc, delta):
    # Extract the last row's values
    last_row = net_df.iloc[-1]
    year = last_row['year']
    net = last_row['net']
    year_change = last_row['year_avg']

    # Initialize a list to store the projected net values
    forecast_df = []

    # fix eval year
    eval_year = year

    # Grow the net value by the yearly average change until project_end
    while year <= project_end:
        year_growth = year_change
        forecast_df.append({'year': year, 'net': net, 'year_change': year_growth})
        net += year_change
        year += 1

    # After project_end, shrink the net value by the yearly change until it hits zero
    while net >= 0 + year_change:
        year_decline = -year_change
        net += year_decline
        forecast_df.append({'year': year, 'net': net, 'year_change': year_decline})
        year += 1

    # Ensure the last value is zero and adjust the last yearly change accordingly
    if net < 0 + year_change:
        remaining_year_decline = -(net % year_change)
        net += remaining_year_decline
        forecast_df.append({'year': year, 'net': net, 'year_change': remaining_year_decline})

    # Convert the list to a DataFrame
    forecast_df = pd.DataFrame(forecast_df)

    # Calculate the scc_discount for each year
    forecast_df['scc_discount'] = forecast_df.apply(
        lambda row: scc.loc[scc['year'] == row['year'], 'central'].values[0] / ((1 + delta) ** (row['year'] - eval_year)),
        axis=1
    )

    return forecast_df

### Define Equivalent Perm Function

The `equivalent_perm` function calculates the equivalent permanent value of the forecasted net additionality. It starts by extracting the first row's net additionality value (`eval_net`) from the `forecast_df` dataframe. This corresponds to the total cumulative additionality from the start of the evaluation period to the end of the evaluation period. Determining a hypothetical permanent value for this additionality (`value_perm`), the value is multiplied by the social cost of carbon in the year at the end of the evaluation period (`eval_scc_discount`):

$$\text{Permanent Value} = (\text{Evaluation\;Year\;Net\;Additionality})\times(\text{Evaluation\;Year\;SCC\;Discount})$$

The function then calculates how many years of draw-down it would take for the evaluation year net additionality to hit zero (by dividing evaluation year net additionality by the evaluation year year on year change). Starting at the last row on `forcast_df` and working backwards across the years corresponding to the years of drawn-down, it mutliplies each negative year change in carbon additionality (negative thus meaning a release of carbon).

$$\text{Draw-Down Period} = \left\lceil \frac{\text{Evaluation Year Net Additionality}}{\text{Evaluation Year Change}} \right\rceil$$

Starting at the last row on `forcast_df` and working backwards across the years corresponding to the years of drawn-down, it mutliplies each negative year change in carbon additionality (negative thus meaning a release of carbon) by that years forecasted social cost of carbon (SCC).
The sum of these negative values corresponds to the damage caused by the emissions:

$$\text{Damage} = \sum_{i=1}^{\text{Draw-Down Years}} \left( \text{Drawn-Down Year Change}_i \times \text{Discounted SCC}_i \right)$$


Due to the discounting of SCC, this damage metric will always be less that the equivalent damage caused if the emissions were released during the evaluation period. Thus, to determine an equivalent permanance, the negative damage is applied to the hypothetical permanent value. This remaining figure represents carbon that has an equivalent permemance. To work out this as a ratio this equivalent permance figure is divided by the hypotheical total permance.

$$\text{Equivalent Permance} = \frac{\text{Permanent Value} + \text{Damage}}{\text{Permanent Value}}$$

In [111]:
def equivalent_perm(forecast_df, project_end):
    # Extract the eval row's net value and yearly change
    eval_row = forecast_df.iloc[0]
    eval_net = eval_row['net']
    print(eval_net)
    eval_year_change = eval_row['year_change']
    print(eval_year_change)
    eval_scc_discount = eval_row['scc_discount']

    # Calculate the number of rows from the end corresponding to the draw-down period
    draw_down_len = math.ceil(eval_net / eval_year_change)
    print(draw_down_len)
    # Calculate the hypothetical permanent value (would also be the damage if all additionality was released in evaluation period)
    value_perm = eval_net * eval_scc_discount
    
    # Calculate the sum of all the yearly change values * their scc discount from the calculated row until the end, counting backwards
    draw_down_index = len(forecast_df) - draw_down_len
    damage_rows = forecast_df.iloc[draw_down_index:]
    damage = damage_rows.apply(
        lambda row: row['year_change'] * row['scc_discount'], axis=1
    ).sum()

    # Calculate E-Perm
    eperm = (value_perm + damage) / value_perm

    return eperm


In [110]:
net_df = net_additionality(additionality, leakage, 5)

proj_df = forecaster(net_df, 2042, scc, 0.03)

ep = equivalent_perm(proj_df, 2042)

print("Equivalent Permanance Ratio =",ep)

497080.5915982798
161561.018484284
4
44
      year            net    year_change  scc_discount
44  2065.0  335519.573114 -161561.018484    128.831853
45  2066.0  173958.554630 -161561.018484    126.930539
46  2067.0   12397.536145 -161561.018484    125.030689
47  2068.0       0.000000  -12397.536145    123.383089
-63050964.39960332
Equivalent Permanance Ratio = 0.4822753504156079
